In [1]:
%load_ext autoreload
%autoreload 2

import sys, os, pickle, time, warnings
import numpy as np, pandas as pd, scipy, scipy.stats as stats, tqdm, h5py
from copy import deepcopy as copy

# Plotting modules
import matplotlib, corner
from pylab import cm
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [2]:
# Plotting modules
font = {'family' : 'serif', 'weight' : 'normal',
        'size'   : 16}
legend = {'fontsize': 16}
matplotlib.rc('font', **font)
matplotlib.rc('legend', **legend)
plt.rc('text', usetex=True)
plt.rc('axes', labelsize=16)
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('legend',fontsize=16)

cm_default = plt.rcParams['axes.prop_cycle'].by_key()['color']

corner_kwargs = {'max_n_ticks':3, 'title_kwargs':{"fontsize": 16}, 'label_kwargs':{'fontsize':16}}

In [3]:
sys.path.extend(['../utilities/', '../models/'])
import samplers, disk_cone_plcut as dcp, plotting, transformations
from transformations import func_inv_jac, func_labels, label_dict
import functions

In [4]:
import disk_halo_mstogap as dh_msto

In [ ]:
pi_mu, pi_err, abs_sin_lat, m_mu, log_pi_err, hz=1., alpha1=-1., alpha2=-1., alpha3=-1.,
                                Mto=4., Mms=8., Mms1=9., Mms2=7., fD=0.5, Mx=10., R0=8.27, degree=21):

# Test Disk Likelihood gradient

In [251]:
size = 1

In [252]:
kwargs = {'hz':1., 'alpha1':-1., 'alpha2':-1., 'alpha3':-1.,
          'Mto':4., 'Mms':8., 'Mms1':9., 'Mms2':7., 
          'fD':0.5, 'Mx':10., 'R0':8.27, 'degree':21}

In [407]:
sample = {}; true_pars={}; latent_pars={}; cmpt=0;
filename = '/data/asfe2/Projects/mwtrace_data/mockmodel/sample.h'
with h5py.File(filename, 'r') as hf:
    subset = (hf['sample']['M'][...]>5)
    subsample  = np.sort(np.random.choice(np.arange(np.sum(subset)), size=size, replace=False))
    for key in hf['sample'].keys():
        sample[key]=hf['sample'][key][...][subset][subsample]
    # Get true parameters
    for key in hf['true_pars'].keys():
        if not key in np.arange(3).astype(str):
            true_pars[key]=hf['true_pars'][key][...]
        else:
            true_pars[int(key)]={}
            for par in hf['true_pars'][key].keys():
                true_pars[int(key)][par]=hf['true_pars'][key][par][...]
for j in range(3): true_pars[j]['w']*=size
    
args = (sample['parallax_obs'], sample['parallax_error'], sample['sinb'], \
    sample['m_obs'], np.log(sample['parallax_error']))
sample

{'M': array([8.12723482]),
 'cmpt': array([1]),
 'l': array([5.86356284]),
 'm': array([21.09319941]),
 'm_err': array([0.47444547]),
 'm_obs': array([20.84397101]),
 'parallax_error': array([0.39568133]),
 'parallax_obs': array([0.28534232]),
 's': array([3.91915933]),
 'sinb': array([0.88310954]),
 'source_id': array([32722])}

In [348]:
dh_msto.log_expmodel_perr(*args, **kwargs), dh_msto.log_expmodel_perr_grad(*args, **kwargs, grad=True)

(array([-4.1757247]), (array([-4.17366804]), array([[ 1.1690662 ],
         [-0.31573264],
         [ 0.28635836],
         [        nan],
         [ 1.46510109],
         [ 0.05013797]])))

In [349]:
kwargs = {'Mms':8., 'Mms1':9., 'Mms2':7., 
          'Mx':10., 'R0':8.27, 'degree':21}
model = lambda x: dh_msto.log_expmodel_perr_grad(*args, hz=x[0], alpha3=x[1], fD=x[2], alpha1=x[4], alpha2=x[5],
                                                Mto=x[3],  **kwargs, grad=True)[0]
grad = lambda x: dh_msto.log_expmodel_perr_grad(*args, hz=x[0], alpha3=x[1], fD=x[2], alpha1=x[4], alpha2=x[5],
                                                Mto=x[3],  **kwargs, grad=True)[1][:,0]

In [353]:
x0 = np.array([1,-0.5,0.5,4,-1,-1.1])
np.vstack(( scipy.optimize.approx_fprime(x0, model, 1e-10), grad(x0) )).T

array([[ 0.30793146,  0.30770291],
       [-0.51718629, -0.51722961],
       [ 0.7872103 ,  0.78721123],
       [ 0.74306783,         nan],
       [ 0.58413718,  0.58426291],
       [ 0.07648993,  0.07648868]])

# Halo

In [354]:
warnings.filterwarnings('ignore', r'divide by zero encountered')

In [408]:
kwargs = {'Mms':8., 'Mms1':9., 'Mms2':7., 
          'Mx':10., 'R0':8.27, 'degree':11}
model = lambda x: dh_msto.log_halomodel_perr_grad(*args, hz=x[0], alpha3=x[1], fD=x[2], alpha1=x[4], alpha2=x[5],
                                                Mto=x[3],  **kwargs, grad=True)[0]
grad = lambda x: dh_msto.log_halomodel_perr_grad(*args, hz=x[0], alpha3=x[1], fD=x[2], alpha1=x[4], alpha2=x[5],
                                                Mto=x[3],  **kwargs, grad=True)[1][:,0]

In [409]:
x0 = np.array([2.,-1.5,0.5,4,-1,-1.1])
np.vstack(( scipy.optimize.approx_fprime(x0, model, 1e-10), grad(x0) )).T

array([[-1.70722991, -1.7065387 ],
       [ 0.31675107,  0.31731099],
       [-1.97395877, -1.97396354],
       [-0.50544458,         nan],
       [ 0.0074607 ,  0.00745997],
       [ 0.00257572,  0.00256529]])

In [386]:
test_args = (1,3,2.,0.5,0.1,0.,0.,1.)
model = lambda h: dh_msto.halomodel_perr_integrand(0.1, *(1,3,h,0.5,0.1))
grad = lambda h: dh_msto.halomodel_perr_integrand_dh(0.1, *(1,3,h,0.5,0.1))

In [387]:
h = 0.1
np.vstack(( scipy.optimize.approx_fprime(np.array([h]), model, 1e-10), grad(h) )).T

array([[-6.14583813e-07, -6.14583826e-07]])

In [383]:
test_args = (1,3,2.,0.5,0.1,0.,0.,1.)
model = lambda n: dh_msto.halomodel_perr_integrand(0.1, *(1,n,2.,0.5,0.1))
grad = lambda n: dh_msto.halomodel_perr_integrand_dn(0.1, *(1,n,2.,0.5,0.1))

In [388]:
n=0.1
np.vstack(( scipy.optimize.approx_fprime(np.array([n]), model, 1e-10), grad(n) )).T

array([[-6.14583813e-07, -6.14583826e-07]])

In [202]:
test_args = (0.11459087188687923, -1.9315154043531053, 62.460138858539736,
             -0.6133993246056172,  1.002678948333817, 0.2, 0.5709469567273955)
beta, n, h, mu, err, a, b = test_args
grad = lambda x: dh_msto.halomodel_perr_logit_grad(x, test_args) \
               * dh_msto.halomodel_perr_integrand(x, *test_args[:-2])/(x*(beta**2+x**2))
model = lambda x: (x-test_args[-2])*(test_args[-1]-x)*dh_msto.halomodel_perr_integrand(x, *test_args[:-2])

In [176]:
model(np.array([0.4, 0.5, 0.3]))

array([0.01025454, 0.00886006, 0.00260297])

In [203]:
p=0.4
scipy.optimize.approx_fprime(np.array([p]), model, 1e-10)[0], grad(p)

(0.052879228773505815, 0.05287882306911542)

In [220]:
grad = lambda x: dh_msto.halomodel_perr_logit_grad_dn(x, test_args) \
               * dh_msto.halomodel_perr_integrand_dn(x, *test_args[:-2])/(x*(beta**2+x**2))
model = lambda x: (x-test_args[-2])*(test_args[-1]-x)*dh_msto.halomodel_perr_integrand_dn(x, *test_args[:-2])

In [216]:
p=0.4
scipy.optimize.approx_fprime(np.array([p]), model, 1e-10)[0], grad(p)

(-0.02281636685141919, -0.022816013578456487)

In [233]:
grad = lambda x: dh_msto.halomodel_perr_logit_grad_dh(x, test_args) \
               * dh_msto.halomodel_perr_integrand_dh(x, *test_args[:-2])/(x*(beta**2+x**2))
model = lambda x: (x-test_args[-2])*(test_args[-1]-x)*dh_msto.halomodel_perr_integrand_dh(x, *test_args[:-2])

In [237]:
p=0.4
scipy.optimize.approx_fprime(np.array([p]), model, 1e-9)[0], grad(p)

(-0.00014103079598246082, -0.00014103021325713786)

# Full multi-component model with logmodel_perr_grad
Data generated in generate_population.py

In [5]:
warnings.filterwarnings('ignore', r'invalid value')
warnings.filterwarnings('ignore', r'divide by zero encountered')

In [166]:
size = 10000

In [167]:
sample = {}; true_pars={}; latent_pars={};
filename = '/data/asfe2/Projects/mwtrace_data/mockmodel/sample.h'
with h5py.File(filename, 'r') as hf:
    subset = (hf['sample']['m'][...]>0)&(hf['sample']['m'][...]<30)
    subsample  = np.sort(np.random.choice(np.arange(np.sum(subset)), size=size, replace=False))
    for key in hf['sample'].keys():
        sample[key]=hf['sample'][key][...][subset][subsample]
    # Get true parameters
    for key in hf['true_pars'].keys():
        if not key in np.arange(3).astype(str): 
            true_pars[key]=hf['true_pars'][key][...]
        else:
            true_pars[key]={}
            for par in hf['true_pars'][key].keys():
                true_pars[key][par]=hf['true_pars'][key][par][...]

In [169]:
np.min(np.vstack((sample['m'], sample['M'])).T, axis=0), np.max(np.vstack((sample['m'], sample['M'])).T, axis=0)

(array([10.41065703, -1.7413033 ]), array([29.98485077, 10.69995811]))

In [170]:
sys.path.extend(['../utilities/', '../models/'])
import samplers, disk_cone_plcut as dcp, plotting, transformations

In [171]:
labels = np.array( [r'$\log(\mathcal{N})$',  r'$\log(1/h_z)$', r'$\alpha$'])

In [172]:
true_pars

{'0': {'Mto': array(4.8),
  'alpha3': array(-1.),
  'fD': array(0.94),
  'hz': array(0.9),
  'w': array(0.2)},
 '1': {'Mto': array(3.14),
  'alpha3': array(-0.5),
  'fD': array(0.998),
  'hz': array(1.9),
  'w': array(0.3)},
 '2': {'Mto': array(3.3),
  'alpha3': array(-0.6),
  'fD': array(0.995),
  'hz': array(4.6),
  'w': array(0.5)},
 'Mms': array(8.),
 'Mms1': array(9.),
 'Mms2': array(7.),
 'Mx': array(10.7),
 'N': array(100000),
 'R0': array(8.27),
 'alpha1': array(-0.15),
 'alpha2': array(-0.3),
 'theta_deg': array(60)}

In [173]:
fid_pars = {'Mmax':true_pars['Mx'],  'lat_min':np.deg2rad(true_pars['theta_deg']),
            'R0':true_pars['R0'],
            'free_pars':{}, 'fixed_pars':{}, 'functions':{}, 'functions_inv':{}, 'jacobians':{}, 'w':True,
            'models':[dh_msto.log_expmodel_grad], 'components':['disk'], 'ncomponents':1}

fid_pars['free_pars'][0] = ['w', 'hz', 'alpha3']
fid_pars['free_pars']['shd'] = []

fid_pars['fixed_pars'][0] = {'Mms':true_pars['Mms'], 'fD':0.001, 'alpha1':-1., 'alpha2':-1.,
                             'Mms1':true_pars['Mms1'], 'Mms2':true_pars['Mms2'], 'Mto':true_pars['0']['Mto']}

print(' Free: ', fid_pars['free_pars'][0], ' Free shared: ', fid_pars['free_pars']['shd'], ' Fixed: ', fid_pars['fixed_pars'][0])

fid_pars['functions'][0] = {'hz':lambda x:np.exp(-x), 'alpha3':lambda x:x, 'w':lambda x: np.exp(x)}
fid_pars['functions_inv'][0] = {'hz':lambda x:np.log(1/x), 'alpha3':lambda x: x, 'w':lambda x: np.log(x)}
fid_pars['jacobians'][0] = {'alpha3':lambda x:x, 'w':lambda x:x,'hz':lambda x:x}

fid_pars['functions'][0] = {'hz':lambda x:x, 'alpha3':lambda x:x, 'w':lambda x:x}
fid_pars['functions_inv'][0] = {'hz':lambda x:x, 'alpha3':lambda x:x, 'w':lambda x:x}
fid_pars['jacobians'][0] = {'alpha3':lambda x:1, 'w':lambda x:1,'hz':lambda x:1}

 Free:  ['w', 'hz', 'alpha3']  Free shared:  []  Fixed:  {'Mms': array(8.), 'alpha1': -1.0, 'alpha2': -1.0, 'Mto': array(4.8), 'fD': 0.001, 'Mms2': array(7.), 'Mms1': array(9.)}


In [174]:
sample_2d = np.vstack((sample['parallax_obs'], sample['parallax_error'], 
                       np.abs(sample['sinb']), np.log(np.sqrt(1-sample['sinb']**2)), 
                       sample['m'], np.log(sample['parallax_error'])))

true_params = np.array([np.log(size), np.log(1/true_pars['0']['hz']), 
                        true_pars['0']['alpha3']])

## Disk

In [175]:
fid_pars['models']=[dh_msto.log_expmodel_perr_grad]

bounds = np.array([[-10, -10,         -5.],
                   [ 10,  10,          0.]])
p0 = np.array([10,1.,-1.])

In [176]:
np.argwhere(np.isnan(dh_msto.logmodel_perr_grad(sample_2d, p0, gmm=None, fid_pars=fid_pars, grad=False)))

array([], shape=(0, 1), dtype=int64)

In [63]:
bad_obj = np.argwhere(np.isnan(dh_msto.logmodel_perr_grad(sample_2d, p0, gmm=None, fid_pars=fid_pars, grad=False)))[0]
bad_obj, dh_msto.logmodel_perr_grad(sample_2d[:,bad_obj], p0, gmm=None, fid_pars=fid_pars, grad=False)

[array([ -7.45154321,  -8.82377175,  -3.7747559 ,  -9.02134549,
       -19.36860884,  -0.37975832,   1.96981603,   3.27091049,
       -15.12808968,  -5.95555861, -13.32974271,  -1.51753746,
        -1.78385877,   1.66326625,  -6.46117314, -16.29585527,
        -0.62830412,   0.34062622,  -4.67048652, -15.13378877,
        -4.79646006, -15.00651411,  -5.61661402,  -0.95270519,
       -21.66171742,   2.35755573,   2.35237379,   3.91154021,
         2.0354968 ,   0.43582452,   2.18540216,  -4.88775394,
         3.40811248,   5.00727462,  -4.49574102,   3.54021793,
         2.3621609 ,   4.07519568,   5.47418678,   2.8296864 ,
         3.89126964,  -6.17364011,   1.10023598,   2.36675875,
         3.85037342,   4.53324454,   4.37735804,  -4.04034609,
       -12.04202663,   5.38087909,  -1.31209065,  -1.37061303,
        -5.03828379,   0.15796514,   2.60166294,   1.604955  ,
         5.1897219 ,   3.6594058 , -10.74497222,   2.87864403,
         4.22678271, -11.01333143,  -3.23534792,   4.3

IndexError: index 0 is out of bounds for axis 0 with size 0

In [177]:
inf_obj = np.argwhere(np.isinf(dh_msto.logmodel_perr_grad(sample_2d, p0, gmm=None, fid_pars=fid_pars, grad=False)))[:,0]
inf_obj

array([], dtype=int64)

In [164]:
sample_2d[:,inf_obj]

array([[ 1.48385339e-02,  1.08477993e+00, -1.78855093e-01,
         3.85837891e-01,  5.55349026e-01,  6.28747845e-03,
         1.09142843e-02,  2.64572829e-01, -5.72440023e-03,
        -3.55236989e-02, -3.84341298e-03, -3.52580763e-02,
        -2.62654380e-01,  1.06557304e+00,  4.39538285e-01,
        -2.72937873e-01,  4.11317500e-02,  1.59562354e-02,
        -5.09257325e-02, -2.24200048e-02, -2.75072465e-02,
        -3.54509110e-02],
       [ 4.46698210e-01,  1.25195337e+00,  2.12044982e-01,
         4.88477538e-01,  3.19728029e-01,  6.84057424e-03,
         2.00298168e-02,  2.56527777e-01,  4.43001437e-01,
         2.05884544e-01,  7.00714997e-02,  7.72309162e-01,
         1.16564467e+00,  7.45795394e-01,  5.53174343e-01,
         3.32488786e-01,  1.61462623e-01,  2.06643341e-02,
         7.27430561e-01,  4.77226645e-02,  2.20010323e-01,
         3.17763260e-01],
       [ 8.82919093e-01,  9.07588307e-01,  9.28398109e-01,
         9.40026252e-01,  9.02133253e-01,  9.06328892e-01,
    

In [ ]:
dh_msto.logmodel_perr_grad(sample_2d, p0, gmm=None, fid_pars=fid_pars, grad=False))

In [537]:
%time dh_msto.logmodel_perr_grad(sample_2d, p0, gmm=None, fid_pars=fid_pars, grad=False)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 19 ms


array([  7.21422587,   4.75754083, -16.59355432,   9.84805218,
         6.25925424])

In [538]:
%time dh_msto.logmodel_perr_grad(sample_2d, p0, gmm=None, fid_pars=fid_pars, grad=True)

CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 30.2 ms


(array([  7.21422587,   4.75754083, -16.59355432,   9.84805218,
          6.25925424]),
 array([[ 2.00048528e+00,  2.31457090e+00,  2.03228025e+00,
          2.45382819e+00,  3.10600756e+00],
        [ 5.72179784e+00,  4.64648160e+00,  1.07688270e+00,
          2.17027318e-02,  1.06453887e+01],
        [-4.28364825e+00, -1.21972029e+01, -5.38827071e+01,
          9.85175411e-01,  2.18449906e+00]]))

In [153]:
source_i=10
model = lambda x: dh_msto.logmodel_perr_grad(sample_2d, x, gmm=None, fid_pars=fid_pars, grad=False)[source_i]
grad  = lambda x: dh_msto.logmodel_perr_grad(sample_2d, x, gmm=None, fid_pars=fid_pars, grad=True)[1][:,source_i]

In [154]:
np.vstack((scipy.optimize.approx_fprime(p0, model, 1e-9), grad(p0))).T

../models/disk_halo_mstogap.py:431: RuntimeWarning: overflow encountered in true_divide
  - 5/ln10 * dalphag_dalpha1 * dp_model_dn[2])))/exp_log_p


array([[ 0.10000001,  0.1       ],
       [-0.8267218 , -0.82674646],
       [-0.01064038, -0.01064181]])

In [178]:
model = lambda x: np.sum(dh_msto.logmodel_perr_grad(sample_2d, x, gmm=None, fid_pars=fid_pars, grad=False))
grad  = lambda x: np.sum(dh_msto.logmodel_perr_grad(sample_2d, x, gmm=None, fid_pars=fid_pars, grad=True)[1], axis=1)

In [180]:
np.vstack((scipy.optimize.approx_fprime(p0, model, 1e-9), grad(p0))).T

array([[  999.99306258,  1000.        ],
       [45497.23234959, 45496.6214292 ],
       [-1089.41458166, -1090.16036926]])

## Halo

In [181]:
fid_pars['models']=[dh_msto.log_halomodel_perr_grad]

bounds = np.array([[-10, -10,         -5.],
                   [ 10,  10,          0.]])
p0 = np.array([10,3.5,-1.])

In [184]:
%time dh_msto.logmodel_perr_grad(sample_2d, p0, gmm=None, fid_pars=fid_pars, grad=False)

CPU times: user 192 ms, sys: 0 ns, total: 192 ms
Wall time: 190 ms


array([ 1.57287378,  1.17778232,  0.30977323, ...,  0.00390195,
       -2.8450074 , -1.59777781])

In [183]:
%time dh_msto.logmodel_perr_grad(sample_2d, p0, gmm=None, fid_pars=fid_pars, grad=True)

CPU times: user 908 ms, sys: 4 ms, total: 912 ms
Wall time: 912 ms


(array([ 1.57287378,  1.17778232,  0.30977323, ...,  0.00390195,
        -2.8450074 , -1.59777781]),
 array([[ 0.1       ,  0.1       ,  0.1       , ...,  0.1       ,
          0.1       ,  0.1       ],
        [ 0.5587805 ,  0.40175467,  1.62551269, ..., -0.61062181,
         -0.71136006,  0.8454847 ],
        [-0.1681301 , -0.14347309,  0.11014306, ..., -0.17834014,
         -0.21274665, -0.17380455]]))

In [185]:
source_i=10
model = lambda x: dh_msto.logmodel_perr_grad(sample_2d[:,:20], x, gmm=None, fid_pars=fid_pars, grad=False)[source_i]
grad  = lambda x: dh_msto.logmodel_perr_grad(sample_2d[:,:20], x, gmm=None, fid_pars=fid_pars, grad=True)[1][:,source_i]

In [186]:
np.vstack((scipy.optimize.approx_fprime(p0, model, 1e-9), grad(p0))).T

array([[ 0.10000001,  0.1       ],
       [ 1.26049571,  1.26030186],
       [-0.13215029, -0.1262598 ]])

In [187]:
model = lambda x: np.sum(dh_msto.logmodel_perr_grad(sample_2d, x, gmm=None, fid_pars=fid_pars, grad=False))
grad  = lambda x: np.sum(dh_msto.logmodel_perr_grad(sample_2d, x, gmm=None, fid_pars=fid_pars, grad=True)[1], axis=1)

In [190]:
np.vstack((scipy.optimize.approx_fprime(p0, model, 1e-10), grad(p0))).T

array([[ 1000.00761449,  1000.        ],
       [ 3294.71731675,  3282.22156212],
       [-1006.91977423,  -931.20621031]])

Gradient is correct but there are some nan bugs in here!